In [1]:
import json
import os
import numpy as np

In [2]:
def get_summarized_result(data):
    data["result"] = np.round(np.mean([res["metric"] for res in data["results"]]), decimals=4)
    data["total_input_tokens"] = sum([res["prompt_tokens"] for res in data["results"]])
    data["total_output_tokens"] = sum([res["completion_tokens"] for res in data["results"]])
    if "test_hash" in data:
        del data["test_hash"]
        del data["seed"]
    del data["result_dir"]
    del data["results"]
    return data

def gather_results(results_dir):
    results = []
    for file in os.walk(results_dir):
        for filename in file[2]:
            # Comment this if you want to include direct results
            if "direct" in filename:
                continue
            with open(os.path.join(file[0], filename), "r") as f:
                data = json.load(f)
                results.append(get_summarized_result(data))
                results[-1]["result_name"] = os.path.join(results_dir, file[0], filename)
    return results

In [6]:
model_dirs = [
    "results_dir/"
]

In [ ]:
all_results = []
for model_dir in model_dirs:
    model_results = gather_results(model_dir)
    all_results += model_results
    print(f"Model {model_dir} done", len(model_results))
    print(len(all_results))

In [ ]:
import csv

# Function to remove newlines from dictionary values
def clean_dict(d):
    return {k: v.replace('\n', ' ') if isinstance(v, str) else v for k, v in d.items()}

# Clean the results
cleaned_results = [clean_dict(result) for result in all_results]

# Write to CSV
keys = cleaned_results[0].keys()
with open('gathered_results.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(cleaned_results)